# TFRecord Builder

In [1]:
import glob
import os
import tensorflow as tf
import cv2

In [2]:
os.listdir()

['Defect_images',
 '.DS_Store',
 'Capstone project evaluation.docx',
 'Untitled.ipynb',
 'Mask_images',
 'dataset',
 'Textile Error Detection Capstone Project.ipynb',
 'Base_Line_Model.ipynb',
 'Capstone_Project_Sang Don Lee_2020-12-01.pptx',
 'NODefect_images',
 '.ipynb_checkpoints',
 'TFRecords.ipynb']

In [29]:
## Define the paths for the files to retrieve and output directory of TFrecords

PASS = 'dataset/3/0/*.png'
FAIL = 'dataset/3/1/*.png'

TFRECORD_PATH = 'tfrecords/'
IMAGE_PER_TFRECORD = 100

In [30]:
pass_list = glob.glob(PASS)
fail_list = glob.glob(FAIL)

In [6]:
## increase the fail list up to the length of PASS file number

## difference the number of files between pass and fail list
diff_times = ok_len//fail_len
remainder = ok_len % fail_len


In [14]:
fail_list_new = []

for _ in range(diff_times):
    fail_list_new = fail_list_new + fail_list

len(fail_list_new)

1074

In [15]:
## check the new fail list length
# print(len(fail_list_new))

## add the remainders 
fail_list_new = fail_list_new + fail_list[: remainder]
fail_list = fail_list_new


## creating labels and combine them together
pass_label = [0] * len(pass_list)
fail_label = [1] * len(fail_list)

file_list = pass_list + fail_list
label_list =  pass_label + fail_label

In [18]:
len(pass_list)

1240

In [17]:
len(fail_list)

1240

In [19]:
## copied from TF website
# The following functions can be used to convert a value to a type compatible
# with tf.train.Example.

def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def image_example(image_string, label):
    image_shape = tf.image.decode_image(image_string).shape

    feature = {
        'height': _int64_feature(image_shape[0]),
        'width': _int64_feature(image_shape[1]),
        'depth': _int64_feature(image_shape[2]),
        'label': _int64_feature(label),
        'image_raw': _bytes_feature(image_string),
    }

    return tf.train.Example(features=tf.train.Features(feature=feature))

## Write TFRecords

In [20]:
if os.path.exists(TFRECORD_PATH) is False:
    os.mkdir(TFRECORD_PATH)

num_tfrecords = len(file_list) // IMAGE_PER_TFRECORD
# if len(file_list) % IMAGE_PER_TFRECORD != 0:
#     num_tfrecords += 1

for idx in range(num_tfrecords):
    idx0 = idx * IMAGE_PER_TFRECORD
    print(idx0)
    idx1 = idx0 + IMAGE_PER_TFRECORD
    print(idx1)
    record_file = TFRECORD_PATH + '%05d.tfrecords' % idx
    with tf.io.TFRecordWriter(record_file) as writer:
        for filename, label in zip(file_list[idx0:idx1],label_list[idx0:idx1]):
            image_string = open(filename, 'rb').read()
            tf_example = image_example(image_string, label)
            writer.write(tf_example.SerializeToString())

0
100
100
200
200
300
300
400
400
500
500
600
600
700
700
800
800
900
900
1000
1000
1100
1100
1200
1200
1300
1300
1400
1400
1500
1500
1600
1600
1700
1700
1800
1800
1900
1900
2000
2000
2100
2100
2200
2200
2300
2300
2400


In [24]:
tffiles = glob.glob('../input/tfrecords/*')
raw_image_dataset = tf.data.TFRecordDataset(tffiles)

image_feature_description = {
    'height': tf.io.FixedLenFeature([], tf.int64),
    'width': tf.io.FixedLenFeature([], tf.int64),
    'depth': tf.io.FixedLenFeature([], tf.int64),
    'label': tf.io.FixedLenFeature([], tf.int64),
    'image_raw': tf.io.FixedLenFeature([], tf.string),
}

def _parse_image_function(example_proto):
    return tf.io.parse_single_example(example_proto, image_feature_description)

def _parse_image_label(parsed_dataset):
    return preprocess(tf.image.decode_png(parsed_dataset['image_raw'])), parsed_dataset['label']

parsed_image_dataset = raw_image_dataset.map(_parse_image_function)

In [25]:
for image_features in parsed_image_dataset:
  image_raw = image_features['image_raw'].numpy()
  display.display(display.Image(data=image_raw))